# Package prepare

In [3]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
import yfinance as yf
import requests
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import datetime as dt
from math import floor
from termcolor import colored as cl
import matplotlib.pyplot as plt
from datetime import datetime
from pandas.core.arrays.datetimelike import timedelta

# Functions

In [42]:
def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []
    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)
    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()
    up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
    down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()
    rs = up_ewm/down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns = {0:'rsi'}).set_index(close.index)
    rsi_df = rsi_df.dropna()
    return rsi_df[1:]

In [43]:
# <30 buy, >70 sell
def implement_rsi_strategy(prices, rsi):    
    buy_price = []
    sell_price = []
    rsi_signal = []
    signal = 0

    for i in range(len(rsi)):
        if rsi[i-1] > 30 and rsi[i] < 30:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
        elif rsi[i-1] < 70 and rsi[i] > 70:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            rsi_signal.append(0)
            
    return buy_price, sell_price, rsi_signal

In [44]:
def calcReturns(df):
    df['returns'] = df['Close'] / df['Close'].shift(1)
    df['log_returns'] = np.log(df['returns'])
    df['strat_returns'] = df['rsi_position'].shift(1) * df['returns']
    df['strat_log_returns'] = df['rsi_position'].shift(1) * df['log_returns']
    df['cum_returns'] = np.exp(df['log_returns'].cumsum()) 
    df['strat_cum_returns'] = np.exp(df['strat_log_returns'].cumsum()) 

In [45]:
def weight_change_fun(start,end,investment,m):
  name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
  df1 = pd.DataFrame()
  df2 = pd.DataFrame()
  df3 = pd.DataFrame()
  df4 = pd.DataFrame()
  df5 = pd.DataFrame()
  df6 = pd.DataFrame()
  df7 = pd.DataFrame()
  df8 = pd.DataFrame()
  df9 = pd.DataFrame()
  df10 = pd.DataFrame()
  d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  start=datetime.strptime(start,'%Y-%m-%d')
  end=datetime.strptime(end,'%Y-%m-%d')
  for i in range(10):
      d2[i] = yf.download(name[i],start = start,end = end)

  for i in range(10):
      d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
      d2[i] = d2[i].dropna()

  buy_price = [0] * 10
  sell_price = [0] * 10
  rsi_signal = [0] * 10
  for i in range(10):
      buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
  strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  for j in range(10):
      position = []
      for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
          if rsi_signal[j][i] > 1:
              position.append(0)
          else:
              position.append(1)
        
      for i in range(len(d2[j]['Close'])):
          if rsi_signal[j][i] == 1:
              position[i] = 1
          elif rsi_signal[j][i] == -1:
              position[i] = 0
          else:
              position[i] = position[i-1]
        
      rsi = d2[j]['rsi_14']
      close_price = d2[j]['Close']
      rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
      position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

      frames = [close_price, rsi, rsi_signal[j], position]
      strategy[j] = pd.concat(frames, join = 'inner', axis = 1) 

  rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  total_investment_ret = [0]*10

  for j in range(10):
      df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
      rsi_strategy_ret = []
    
      for i in range(len(df_ret)):
          returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
          rsi_strategy_ret.append(returns)
    
      rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
      investment_value = investment # 每个公司weight变化后的n投资(n>1)
      number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
      rsi_investment_ret = []
    
      for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
          returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
          rsi_investment_ret.append(returns)
    
      rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
      total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

  rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  for i in range(10):
    rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

  for i in range(10):
    strategy[i]=strategy[i][1:]

  for i in range(10):
    rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

  rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  for i in range(10):
    rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

  totalreturn=[]
  for i in range(10):
    totalreturn.append(round(sum(rsi_dreturn[i]['investment_returns'])/investment_value,4))

  current_investment=[]
  for i in range(10):
    current_investment.append(round(investment_value+sum(rsi_dreturn[i]['investment_returns']),2))

  return current_investment[m]

In [46]:
name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
    d2[i] = yf.download(name[i],start = '2019-01-01',end = '2019-12-31')

for i in range(10):
    d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
    d2[i] = d2[i].dropna()

buy_price = [0] * 10
sell_price = [0] * 10
rsi_signal = [0] * 10
for i in range(10):
    buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for j in range(10):
    position = []
    for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
        if rsi_signal[j][i] > 1:
            position.append(0)
        else:
            position.append(1)
        
    for i in range(len(d2[j]['Close'])):
        if rsi_signal[j][i] == 1:
            position[i] = 1
        elif rsi_signal[j][i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]
        
    rsi = d2[j]['rsi_14']
    close_price = d2[j]['Close']
    rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
    position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

    frames = [close_price, rsi, rsi_signal[j], position]
    strategy[j] = pd.concat(frames, join = 'inner', axis = 1) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [47]:
strategy[0]

,Close,rsi_14,rsi_signal,rsi_position
Date,,,,
2019-01-04,172.259995,60.792466,0,1
2019-01-07,173.639999,64.820788,0,1
2019-01-08,175.229996,68.798475,0,1
2019-01-09,173.660004,61.414329,0,1
2019-01-10,177.149994,69.301916,0,1
...,...,...,...,...
2019-12-23,271.369995,58.527346,0,0
2019-12-24,272.500000,60.515105,0,0
2019-12-26,273.559998,62.338613,0,0


In [48]:
weights = [0.1, 0.1, 0.1, 0.1, 0.1 ,0.1, 0.1, 0.1, 0.1, 0.1]

rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
total_investment_ret = [0]*10
for j in range(10):
    df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
    rsi_strategy_ret = []
    
    for i in range(len(df_ret)):
        returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
        rsi_strategy_ret.append(returns)
    
    rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
    investment_value = 10000 # total investment=100000, equally weighted, each invest 10000.
    number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
    rsi_investment_ret = []
    
    for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
        returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
        rsi_investment_ret.append(returns)
    
    rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)



In [49]:
total_investment_ret[0]

542.88

In [50]:
rsi_investment_ret_df[0]

,investment_returns
0,80.040283
1,92.219788
2,-91.059540
3,202.419434
4,-67.859894
...,...
243,-0.000000
244,0.000000
245,0.000000
246,0.000000


In [51]:
rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

In [52]:
rsi_daily_return[0]

,investment_returns,rsi_returns
0,80.040283,1.380005
1,92.219788,1.589996
2,-91.059540,-1.569992
3,202.419434,3.489990
4,-67.859894,-1.169998
...,...,...
243,-0.000000,-0.000000
244,0.000000,0.000000
245,0.000000,0.000000
246,0.000000,0.000000


In [53]:
for i in range(10):
  strategy[i]=strategy[i][1:]

In [54]:
for i in range(10):
  rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

rsi_dreturn_2019=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn_2019[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

In [55]:
rsi_dreturn_2019[0]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-07,173.639999,64.820788,0,1,80.040283,1.380005
2019-01-08,175.229996,68.798475,0,1,92.219788,1.589996
2019-01-09,173.660004,61.414329,0,1,-91.059540,-1.569992
2019-01-10,177.149994,69.301916,0,1,202.419434,3.489990
2019-01-11,175.979996,64.538867,0,1,-67.859894,-1.169998
...,...,...,...,...,...,...
2019-12-23,271.369995,58.527346,0,0,-0.000000,-0.000000
2019-12-24,272.500000,60.515105,0,0,0.000000,0.000000
2019-12-26,273.559998,62.338613,0,0,0.000000,0.000000


In [56]:
totalreturn_2019=[]
for i in range(10):
  totalreturn_2019.append(round(sum(rsi_dreturn_2019[i]['investment_returns'])/investment_value,4))

totalreturn_2019

[0.0543, 0.0, 0.1121, 0.0, 0.2305, 0.1748, 0.5947, -0.0872, 0.2123, 0.2767]

In [57]:
#end remaining money, is also the investment value for the next period
current_investment=[]
for i in range(10):
  current_investment.append(round(investment_value+sum(rsi_dreturn_2019[i]['investment_returns']),2))
current_investment

[10542.88,
 10000.0,
 11121.16,
 10000.0,
 12305.03,
 11748.46,
 15946.92,
 9127.8,
 12123.13,
 12766.68]

In [58]:
sum(current_investment)/100000

1.1568206

In [59]:
# calculate new investment weight
totalchange = sum(np.absolute(totalreturn_2019))
new_weight_change=[]
for i in range(10):
  new_weight_change.append(round(totalreturn_2019[i]/totalchange,4))

new_weights_1=[]
for i in range(10):
  new_weights_1.append(round(weights[i]*(1 + new_weight_change[i]),2))

total=sum(new_weights_1)
new_weights=[]
for i in range(10):
  new_weights.append(round(new_weights_1[i]/total,2))

print('new_weights is',new_weights)

new_weights is [0.09, 0.09, 0.1, 0.09, 0.1, 0.1, 0.12, 0.09, 0.1, 0.11]


In [60]:
strategy[0]

,Close,rsi_14,rsi_signal,rsi_position
Date,,,,
2019-01-07,173.639999,64.820788,0,1
2019-01-08,175.229996,68.798475,0,1
2019-01-09,173.660004,61.414329,0,1
2019-01-10,177.149994,69.301916,0,1
2019-01-11,175.979996,64.538867,0,1
...,...,...,...,...
2019-12-23,271.369995,58.527346,0,0
2019-12-24,272.500000,60.515105,0,0
2019-12-26,273.559998,62.338613,0,0


In [61]:
new_investment=sum(current_investment)
new_investment

115682.06

# 2019 with equal weight

In [62]:
equal_weight_return_2019=[]
for i in range(10):
  equal_weight_return_2019.append(weight_change_fun(start='2019-01-01',end='2020-01-01',investment=10000,m=i))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [63]:
equal_weight_return_2019

[10542.88,
 10000.0,
 11121.16,
 10000.0,
 12305.03,
 11748.46,
 15946.92,
 9169.8,
 12123.13,
 12766.68]

In [64]:
# total return for 2019 with equal weight
new_weight_change=[]
for i in range(10):
  new_weight_change.append(equal_weight_return_2019[i]/sum(equal_weight_return_2019))

new_weight_unscale=[]
for i in range(10):
  new_weight_unscale.append(0.1*(1+new_weight_change[i]))

new_weights=[]
for i in range(10):
  new_weights.append(round(new_weight_unscale[i]/sum(new_weight_unscale),2))

new_weights

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [67]:
new_investment=sum(equal_weight_return_2019)
new_investment

115724.06

# total data from 2019-01-01

In [68]:
name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
    d2[i] = yf.download(name[i],start = '2019-01-01')

for i in range(10):
    d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
    d2[i] = d2[i].dropna()

buy_price = [0] * 10
sell_price = [0] * 10
rsi_signal = [0] * 10
for i in range(10):
    buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for j in range(10):
    position = []
    for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
        if rsi_signal[j][i] > 1:
            position.append(0)
        else:
            position.append(1)
        
    for i in range(len(d2[j]['Close'])):
        if rsi_signal[j][i] == 1:
            position[i] = 1
        elif rsi_signal[j][i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]
        
    rsi = d2[j]['rsi_14']
    close_price = d2[j]['Close']
    rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
    position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

    frames = [close_price, rsi, rsi_signal[j], position]
    strategy[j] = pd.concat(frames, join = 'inner', axis = 1)

rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
total_investment_ret = [0]*10
for j in range(10):
    df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
    rsi_strategy_ret = []
    
    for i in range(len(df_ret)):
        returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
        rsi_strategy_ret.append(returns)
    
    rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
    investment_value = 10000 # total investment=100000, equally weighted, each invest 10000.
    number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
    rsi_investment_ret = []
    
    for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
        returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
        rsi_investment_ret.append(returns)
    
    rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

for i in range(10):
  strategy[i]=strategy[i][1:]

for i in range(10):
  rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

rsi_dreturn_whole=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn_whole[i]=rsi_dreturn[i]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# find the investment period

In [69]:
#提取投资区间
investment_period=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  investment_period[i]=rsi_dreturn_whole[i][((rsi_dreturn_whole[i]['rsi_signal']==1) & (rsi_dreturn_whole[i]['rsi_position']==1)) | ((rsi_dreturn_whole[i]['rsi_signal']==-1) & (rsi_dreturn_whole[i]['rsi_position']==0))]

In [70]:
#start=datetime.strptime('2020-02-25','%Y-%m-%d')
#end=datetime.strptime('2020-05-30','%Y-%m-%d')
#rsi_dreturn_whole[0][(rsi_dreturn_whole[0].index>=start) & (rsi_dreturn_whole[0].index<end)]

In [71]:
rsi_dreturn_whole[0]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-07,173.639999,64.820788,0,1,80.040283,1.380005
2019-01-08,175.229996,68.798475,0,1,92.219788,1.589996
2019-01-09,173.660004,61.414329,0,1,-91.059540,-1.569992
2019-01-10,177.149994,69.301916,0,1,202.419434,3.489990
2019-01-11,175.979996,64.538867,0,1,-67.859894,-1.169998
...,...,...,...,...,...,...
2022-11-18,345.869995,60.355248,0,1,-52.199646,-0.899994
2022-11-21,347.420013,61.090164,0,1,89.901062,1.550018
2022-11-22,352.250000,63.368907,0,1,280.139221,4.829987


# 2020

## SPGI 2020

In [72]:
investment_period[0]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-15,181.619995,74.288240,-1,0,266.799469,4.599991
2020-02-27,267.149994,28.748456,1,1,-0.000000,-0.000000
2020-05-29,325.019989,71.374777,-1,0,360.760071,6.220001
2020-10-27,330.850006,29.471794,1,1,-0.000000,-0.000000
2021-03-29,361.739990,70.726868,-1,0,127.020142,2.190002
2021-10-04,415.850006,29.842482,1,1,-0.000000,-0.000000
2022-08-10,393.200012,72.722919,-1,0,934.380920,16.110016
2022-09-22,322.369995,25.556514,1,1,-0.000000,-0.000000


### (2020-02-27,2020-05-29)

In [73]:
name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
    d2[i] = yf.download(name[i],start = '2020-02-25',end = '2020-05-30')######

for i in range(10):
    d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
    d2[i] = d2[i].dropna()

buy_price = [0] * 10
sell_price = [0] * 10
rsi_signal = [0] * 10
for i in range(10):
    buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for j in range(10):
    position = []
    for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
        if rsi_signal[j][i] > 1:
            position.append(0)
        else:
            position.append(1)
        
    for i in range(len(d2[j]['Close'])):
        if rsi_signal[j][i] == 1:
            position[i] = 1
        elif rsi_signal[j][i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]
        
    rsi = d2[j]['rsi_14']
    close_price = d2[j]['Close']
    rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
    position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

    frames = [close_price, rsi, rsi_signal[j], position]
    strategy[j] = pd.concat(frames, join = 'inner', axis = 1)

 



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [74]:
rsi_signal[0]

,rsi_signal
Date,
2020-02-27,1
2020-02-28,0
2020-03-02,0
2020-03-03,0
2020-03-04,0
...,...
2020-05-22,0
2020-05-26,0
2020-05-27,0


In [75]:
rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
total_investment_ret = [0]*10
for j in range(10):
    df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
    rsi_strategy_ret = []
    
    for i in range(len(df_ret)):
        returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
        rsi_strategy_ret.append(returns)
    
    rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
    investment_value = new_investment*new_weights[0] ########## 每个公司weight变化后的第一次投资,n>1则是current_investment
    number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
    rsi_investment_ret = []
    
    for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
        returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
        rsi_investment_ret.append(returns)
    
    rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

for i in range(10):
  strategy[i]=strategy[i][1:]

for i in range(10):
  rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

totalreturn=[]
for i in range(10):
  totalreturn.append(round(sum(rsi_dreturn[i]['investment_returns'])/investment_value,4))

current_investment=[]
for i in range(10):
  current_investment.append(round(investment_value+sum(rsi_dreturn[i]['investment_returns']),2))

rsi_dreturn_spgi_1=rsi_dreturn[0]###########
current_investment=current_investment[0]#########
current_investment############

14060.82

In [76]:
rsi_dreturn_spgi_1

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2020-02-28,265.910004,9.692559,0,1,-53.319580,-1.239990
2020-03-02,289.309998,63.071154,0,1,1006.199738,23.399994
2020-03-03,280.140015,50.479190,0,1,-394.309265,-9.169983
2020-03-04,296.059998,63.939458,0,1,684.559265,15.919983
2020-03-05,278.420013,48.280088,0,1,-758.519318,-17.639984
...,...,...,...,...,...,...
2020-05-22,311.859985,64.637685,0,1,47.298950,1.099976
2020-05-26,312.339996,64.904502,0,1,20.640472,0.480011
2020-05-27,315.429993,66.649004,0,1,132.869843,3.089996


### (2020-10-27,2020-12-31)

In [77]:
name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
    d2[i] = yf.download(name[i],start = '2020-10-23',end = '2021-01-01')

for i in range(10):
    d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
    d2[i] = d2[i].dropna()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [78]:
d2[0]

,Open,High,Low,Close,Adj Close,Volume,rsi_14
Date,,,,,,,
2020-10-27,330.000000,336.029999,328.850006,330.850006,324.690002,1482700,0.000000
2020-10-28,325.010010,329.570007,322.119995,326.700012,320.617218,1505600,0.000000
2020-10-29,327.410004,329.410004,319.510010,325.739990,319.675018,876400,0.000000
2020-10-30,323.459991,329.779999,319.040009,322.730011,316.721130,1464400,0.000000
2020-11-02,328.570007,335.500000,325.769989,327.920013,321.814545,1305100,30.853522
2020-11-03,331.970001,337.880005,329.570007,334.750000,328.517365,1186600,51.890137
2020-11-04,338.290009,357.690002,338.079987,351.100006,344.562958,1603700,73.037294
2020-11-05,358.059998,360.850006,352.660004,355.989990,349.361816,1317500,76.381173
2020-11-06,355.470001,356.790009,348.119995,354.359985,347.762238,1072300,73.125635


In [79]:

buy_price = [0] * 10
sell_price = [0] * 10
rsi_signal = [0] * 10
for i in range(10):
    buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for j in range(10):
    position = []
    for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
        if rsi_signal[j][i] > 1:
            position.append(0)
        else:
            position.append(1)
        
    for i in range(len(d2[j]['Close'])):
        if rsi_signal[j][i] == 1:
            position[i] = 1
        elif rsi_signal[j][i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]
        
    rsi = d2[j]['rsi_14']
    close_price = d2[j]['Close']
    rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
    position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

    frames = [close_price, rsi, rsi_signal[j], position]
    strategy[j] = pd.concat(frames, join = 'inner', axis = 1) 

rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
total_investment_ret = [0]*10

for j in range(10):
    df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
    rsi_strategy_ret = []
    
    for i in range(len(df_ret)):
        returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
        rsi_strategy_ret.append(returns)
    
    rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
    investment_value = current_investment # 每个公司weight变化后的n投资(n>1)
    number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
    rsi_investment_ret = []
    
    for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
        returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
        rsi_investment_ret.append(returns)
    
    rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

for i in range(10):
  strategy[i]=strategy[i][1:]

for i in range(10):
  rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

totalreturn=[]
for i in range(10):
  totalreturn.append(round(sum(rsi_dreturn[i]['investment_returns'])/investment_value,4))

current_investment=[]
for i in range(10):
  current_investment.append(round(investment_value+sum(rsi_dreturn[i]['investment_returns']),2))

rsi_dreturn_spgi_2=rsi_dreturn[0]
remain_spgi_2020=current_investment[0]
remain_spgi_2020

14911.32

## VZ 2020

In [80]:
investment_period[1]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2020-02-27,55.060001,24.960978,1,1,-0.000000,-0.000000
2020-09-02,60.529999,73.558667,-1,0,242.489811,1.369999
2020-10-19,57.299999,29.371152,1,1,-0.000000,-0.000000


### (2020-02-27,2020-9-02)

In [81]:
name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
    d2[i] = yf.download(name[i],start = '2020-02-25',end = '2020-09-03')

for i in range(10):
    d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
    d2[i] = d2[i].dropna()

buy_price = [0] * 10
sell_price = [0] * 10
rsi_signal = [0] * 10
for i in range(10):
    buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for j in range(10):
    position = []
    for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
        if rsi_signal[j][i] > 1:
            position.append(0)
        else:
            position.append(1)
        
    for i in range(len(d2[j]['Close'])):
        if rsi_signal[j][i] == 1:
            position[i] = 1
        elif rsi_signal[j][i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]
        
    rsi = d2[j]['rsi_14']
    close_price = d2[j]['Close']
    rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
    position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

    frames = [close_price, rsi, rsi_signal[j], position]
    strategy[j] = pd.concat(frames, join = 'inner', axis = 1) 

rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
total_investment_ret = [0]*10

for j in range(10):
    df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
    rsi_strategy_ret = []
    
    for i in range(len(df_ret)):
        returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
        rsi_strategy_ret.append(returns)
    
    rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
    investment_value = new_investment*new_weights[1] # 每个公司weight变化后的n投资(n>1)
    number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
    rsi_investment_ret = []
    
    for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
        returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
        rsi_investment_ret.append(returns)
    
    rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

for i in range(10):
  strategy[i]=strategy[i][1:]

for i in range(10):
  rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

totalreturn=[]
for i in range(10):
  totalreturn.append(round(sum(rsi_dreturn[i]['investment_returns'])/investment_value,4))

current_investment=[]
for i in range(10):
  current_investment.append(round(investment_value+sum(rsi_dreturn[i]['investment_returns']),2))

rsi_dreturn_vz_1=rsi_dreturn[1]
current_investment=current_investment[1]
current_investment

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


12721.11

### (2020-10-19,2020-12-31)

In [82]:
name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
    d2[i] = yf.download(name[i],start = '2020-10-17',end = '2021-01-01')

for i in range(10):
    d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
    d2[i] = d2[i].dropna()

buy_price = [0] * 10
sell_price = [0] * 10
rsi_signal = [0] * 10
for i in range(10):
    buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for j in range(10):
    position = []
    for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
        if rsi_signal[j][i] > 1:
            position.append(0)
        else:
            position.append(1)
        
    for i in range(len(d2[j]['Close'])):
        if rsi_signal[j][i] == 1:
            position[i] = 1
        elif rsi_signal[j][i] == -1:
            position[i] = 0
        else:
            position[i] = position[i-1]
        
    rsi = d2[j]['rsi_14']
    close_price = d2[j]['Close']
    rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
    position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

    frames = [close_price, rsi, rsi_signal[j], position]
    strategy[j] = pd.concat(frames, join = 'inner', axis = 1) 

rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
total_investment_ret = [0]*10

for j in range(10):
    df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
    rsi_strategy_ret = []
    
    for i in range(len(df_ret)):
        returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
        rsi_strategy_ret.append(returns)
    
    rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
    investment_value = current_investment # 每个公司weight变化后的n投资(n>1)
    number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
    rsi_investment_ret = []
    
    for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
        returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
        rsi_investment_ret.append(returns)
    
    rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
    total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

for i in range(10):
  strategy[i]=strategy[i][1:]

for i in range(10):
  rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
for i in range(10):
  rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

totalreturn=[]
for i in range(10):
  totalreturn.append(round(sum(rsi_dreturn[i]['investment_returns'])/investment_value,4))

current_investment=[]
for i in range(10):
  current_investment.append(round(investment_value+sum(rsi_dreturn[i]['investment_returns']),2))

rsi_dreturn_vz_2=rsi_dreturn[1]
remain_vz_2020=current_investment[1]
remain_vz_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


12992.15

## TSLA 2020

### function

In [83]:
def weight_change_fun(start,end,investment,m):
  name = ['SPGI','VZ','TSLA','PG','DIS','SYY','AAPL','XOM','BRK-B','UNH']
  df1 = pd.DataFrame()
  df2 = pd.DataFrame()
  df3 = pd.DataFrame()
  df4 = pd.DataFrame()
  df5 = pd.DataFrame()
  df6 = pd.DataFrame()
  df7 = pd.DataFrame()
  df8 = pd.DataFrame()
  df9 = pd.DataFrame()
  df10 = pd.DataFrame()
  d2 = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  start=datetime.strptime(start,'%Y-%m-%d')
  end=datetime.strptime(end,'%Y-%m-%d')
  for i in range(10):
      d2[i] = yf.download(name[i],start = start,end = end)

  for i in range(10):
      d2[i]['rsi_14'] = get_rsi(d2[i]['Close'], 14)
      d2[i] = d2[i].dropna()

  buy_price = [0] * 10
  sell_price = [0] * 10
  rsi_signal = [0] * 10
  for i in range(10):
      buy_price[i], sell_price[i], rsi_signal[i] = implement_rsi_strategy(d2[i]['Close'], d2[i]['rsi_14'])

# creating position
  strategy = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  for j in range(10):
      position = []
      for i in range(len(rsi_signal[j])): # create position = 1 to assume we hold the stock from the begining.
          if rsi_signal[j][i] > 1:
              position.append(0)
          else:
              position.append(1)
        
      for i in range(len(d2[j]['Close'])):
          if rsi_signal[j][i] == 1:
              position[i] = 1
          elif rsi_signal[j][i] == -1:
              position[i] = 0
          else:
              position[i] = position[i-1]
        
      rsi = d2[j]['rsi_14']
      close_price = d2[j]['Close']
      rsi_signal[j] = pd.DataFrame(rsi_signal[j]).rename(columns = {0:'rsi_signal'}).set_index(d2[j].index)
      position = pd.DataFrame(position).rename(columns = {0:'rsi_position'}).set_index(d2[j].index)

      frames = [close_price, rsi, rsi_signal[j], position]
      strategy[j] = pd.concat(frames, join = 'inner', axis = 1) 

  rsi_strategy_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  rsi_investment_ret_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  total_investment_ret = [0]*10

  for j in range(10):
      df_ret = pd.DataFrame(np.diff(strategy[j]['Close'])).rename(columns = {0:'returns'})
      rsi_strategy_ret = []
    
      for i in range(len(df_ret)):
          returns = df_ret['returns'][i]*(strategy[j]['rsi_position'][i])#gain/loss
          rsi_strategy_ret.append(returns)
    
      rsi_strategy_ret_df[j] = pd.DataFrame(rsi_strategy_ret).rename(columns = {0:'rsi_returns'})

    # calculate investement return 
      investment_value = investment # 每个公司weight变化后的n投资(n>1)
      number_of_stocks = floor(investment_value/(strategy[j]['Close'][0]))
      rsi_investment_ret = []
    
      for i in range(len(rsi_strategy_ret_df[j]['rsi_returns'])):
          returns = number_of_stocks*rsi_strategy_ret_df[j]['rsi_returns'][i]
          rsi_investment_ret.append(returns)
    
      rsi_investment_ret_df[j] = pd.DataFrame(rsi_investment_ret).rename(columns = {0:'investment_returns'})
      total_investment_ret[j] = round(sum(rsi_investment_ret_df[j]['investment_returns']), 2)

  rsi_daily_return=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  for i in range(10):
    rsi_daily_return[i]=pd.merge(rsi_investment_ret_df[i],rsi_strategy_ret_df[i],how='outer',left_index=True,right_index=True)

  for i in range(10):
    strategy[i]=strategy[i][1:]

  for i in range(10):
    rsi_daily_return[i]=rsi_daily_return[i].set_index(strategy[i].index)

  rsi_dreturn=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
  for i in range(10):
    rsi_dreturn[i]=pd.merge(strategy[i],rsi_daily_return[i],how='outer',left_index=True,right_index=True)

  totalreturn=[]
  for i in range(10):
    totalreturn.append(round(sum(rsi_dreturn[i]['investment_returns'])/investment_value,4))

  current_investment=[]
  for i in range(10):
    current_investment.append(round(investment_value+sum(rsi_dreturn[i]['investment_returns']),2))

  return current_investment[m]

In [84]:
investment_period[2]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-07,22.330667,79.856424,-1,0,543.430008,1.151335
2019-04-26,15.676000,28.409089,1,1,-0.000000,-0.000000
2019-07-15,16.900000,72.251919,-1,0,264.949036,0.561333
2020-03-16,29.671333,28.443505,1,1,-0.000000,-0.000000
2020-06-08,63.327999,71.675212,-1,0,2022.048187,4.284000
2021-03-05,199.316666,29.576555,1,1,-0.000000,-0.000000
2021-10-14,272.773346,71.689003,-1,0,1139.106201,2.413361
2022-05-12,242.666672,29.977869,1,1,-0.000000,-0.000000
2022-07-29,297.149994,70.890875,-1,0,7670.000000,16.250000


### (2020-03-16,2020-06-08)

In [85]:
remain_tsla_2020=weight_change_fun(start='2020-03-14',end='2020-06-09',investment=new_investment*new_weights[2],m=2)
remain_tsla_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


17269.37

## PG 2020

In [86]:
investment_period[3]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-02-01,97.470001,72.406282,-1,0,108.000000,1.000000
2020-02-27,113.500000,22.971483,1,1,-0.000000,-0.000000
2020-07-28,127.879997,72.791851,-1,0,168.479736,1.559998
2021-01-20,131.929993,29.784349,1,1,-0.000000,-0.000000
2021-03-26,135.449997,71.220915,-1,0,212.760132,1.970001
2022-05-19,141.699997,27.739337,1,1,-0.000000,-0.000000
2022-11-21,144.380005,72.249222,-1,0,195.479736,1.809998


In [87]:
remain_pg_2020=weight_change_fun(start='2020-02-25',end='2020-07-29',investment=new_investment*new_weights[3],m=3)
remain_pg_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13024.79

## DIS 2020

In [88]:
investment_period[4]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-09,112.669998,73.677686,-1,0,113.750000,1.250000
2019-03-25,107.790001,29.645158,1,1,-0.000000,-0.000000
2019-04-12,130.059998,84.990767,-1,0,1224.859917,13.459999
2020-01-27,135.899994,26.237696,1,1,-0.000000,-0.000000
2020-08-05,127.610001,70.946345,-1,0,939.119972,10.320000
2021-05-14,173.699997,29.719846,1,1,-0.000000,-0.000000
2022-08-11,117.690002,73.052871,-1,0,478.660194,5.260002
2022-09-23,99.500000,29.767402,1,1,-0.000000,-0.000000


In [89]:
remain_dis_2020=weight_change_fun(start='2020-01-25',end='2020-08-06',investment=new_investment*new_weights[4],m=4)
remain_dis_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


10854.16

## SYY 2020

In [90]:
investment_period[5]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-08,62.779999,70.229303,-1,0,157.779926,0.980000
2019-08-05,67.139999,26.996509,1,1,-0.000000,-0.000000
2019-09-11,77.070000,70.819733,-1,0,51.519951,0.320000
2020-02-03,76.680000,23.361603,1,1,-0.000000,-0.000000
2020-09-15,65.400002,70.107743,-1,0,133.630295,0.830002
2020-10-28,54.790001,28.180504,1,1,-0.000000,-0.000000
2020-11-10,72.279999,70.231527,-1,0,193.199509,1.199997
2021-06-17,76.040001,29.947013,1,1,-0.000000,-0.000000
2022-02-15,84.349998,70.247922,-1,0,379.960098,2.360001


In [91]:
current_investment=weight_change_fun(start='2020-02-01',end='2020-09-16',investment=new_investment*new_weights[5],m=5)
current_investment
remain_syy_2020=weight_change_fun(start='2020-10-26',end='2020-11-11',investment=current_investment,m=5)
remain_syy_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

12971.59

## AAPL 2020

In [92]:
investment_period[6]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-02-04,42.812500,71.473357,-1,0,318.092213,1.182499
2019-05-31,43.767502,29.379638,1,1,-0.000000,-0.000000
2019-10-21,60.127499,72.186748,-1,0,275.724384,1.024998
2020-02-27,68.379997,28.656224,1,1,-0.000000,-0.000000
2020-06-05,82.875000,70.958079,-1,0,617.354507,2.294998
2021-02-25,120.989998,28.279096,1,1,-0.000000,-0.000000
2021-06-30,136.960007,70.362899,-1,0,169.471313,0.630005
2022-09-30,138.199997,28.534189,1,1,-0.000000,-0.000000


In [93]:
remain_appl_2020=weight_change_fun(start='2020-02-25',end='2020-06-06',investment=new_investment*new_weights[6],m=6)
remain_appl_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


14022.06

## XOM 2020

In [94]:
investment_period[7]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-05-13,75.709999,29.374817,1,1,-0.000000,-0.000000
2020-06-05,53.080002,71.434699,-1,0,557.200470,3.980003
2020-09-10,37.000000,26.736427,1,1,-0.000000,-0.000000
2020-11-24,41.980000,70.398831,-1,0,366.799850,2.619999
2021-07-19,55.349998,27.134569,1,1,-0.000000,-0.000000
2021-10-20,63.849998,70.118066,-1,0,48.999786,0.349998
2022-09-26,83.980003,29.587907,1,1,-0.000000,-0.000000
2022-10-28,110.699997,72.700543,-1,0,440.999146,3.149994


In [95]:
remain_xom_2020=weight_change_fun(start='2020-09-08',end='2020-11-25',investment=new_investment*new_weights[7],m=7)
remain_xom_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13126.17

## BRK-B 2020

In [96]:
investment_period[8]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-04-29,216.580002,72.006712,-1,0,120.360031,2.360001
2019-08-05,197.729996,28.590942,1,1,-0.000000,-0.000000
2019-11-04,217.979996,71.961118,-1,0,109.649689,2.149994
2020-02-27,207.669998,23.143667,1,1,-0.000000,-0.000000
2020-06-05,200.660004,71.276512,-1,0,439.110031,8.610001
2022-05-02,318.190002,28.550417,1,1,-0.000000,-0.000000
2022-11-23,316.179993,70.008648,-1,0,21.419128,0.419983


In [97]:
remain_brkb_2020=weight_change_fun(start='2020-02-25',end='2020-06-05',investment=new_investment*new_weights[8],m=8)
remain_brkb_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


10713.31

## UNH 2020

In [98]:
investment_period[9]

,Close,rsi_14,rsi_signal,rsi_position,investment_returns,rsi_returns
Date,,,,,,
2019-01-15,256.869995,82.236462,-1,0,361.209900,8.809998
2019-02-28,242.220001,28.120447,1,1,-0.000000,-0.000000
2019-07-12,265.899994,72.836381,-1,0,194.339600,4.739990
2019-08-27,222.929993,25.871952,1,1,-0.000000,-0.000000
2019-10-22,249.479996,70.886259,-1,0,227.959900,5.559998
2020-02-27,253.919998,28.679241,1,1,-0.000000,-0.000000
2021-03-19,365.579987,71.985548,-1,0,144.729950,3.529999
2021-10-04,387.010010,29.377683,1,1,-0.000000,-0.000000
2021-10-21,442.279999,71.337539,-1,0,294.790100,7.190002


In [99]:
remain_unh_2020=weight_change_fun(start='2020-02-25',end='2021-01-01',investment=new_investment*new_weights[9],m=9)
remain_unh_2020

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


12435.96

## return and update weight

In [100]:
# p/l list
com_pl_2020=[remain_spgi_2020,
            remain_vz_2020,
            remain_tsla_2020,
            remain_pg_2020,
            remain_dis_2020,
            remain_syy_2020,
            remain_appl_2020,
            remain_xom_2020,
            remain_brkb_2020,
            remain_unh_2020]
com_pl_2020

[14911.32,
 12992.15,
 17269.37,
 13024.79,
 10854.16,
 12971.59,
 14022.06,
 13126.17,
 10713.31,
 12435.96]

In [101]:
total_return_2020=sum(com_pl_2020)
total_return_2020

132320.87999999998

In [102]:
# return for each company
rcom_2020=[]
for i in range(10):
  rcom_2020.append(round((com_pl_2020[i]-new_investment*new_weights[i])/new_investment*new_weights[i],4))

rcom_2020

[0.0029,
 0.0012,
 0.0049,
 0.0013,
 -0.0006,
 0.0012,
 0.0021,
 0.0013,
 -0.0007,
 0.0007]

In [103]:
# weight changed by return percentage
new_weights_2021_1=[]
for i in range(10):
  new_weights_2021_1.append(new_weights[i]*(1+rcom_2020[i]))
# rescaling
new_weights_2021=[]
for i in range(10):
  new_weights_2021.append(round(new_weights_2021_1[i]/sum(new_weights_2021_1),2))
new_weights_2021

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [104]:
new_investment_2021=total_return_2020
print('the investment for year 2021 is',new_investment_2021)
r20192020=(new_investment_2021-new_investment)/new_investment
print('the return for year 2020 is',r20192020 )

the investment for year 2021 is 132320.87999999998
the return for year 2020 is 0.1434171943155121


# 2021

## SPGI

In [105]:
current_investment=weight_change_fun(start='2020-12-31',end='2021-3-30',investment=new_investment_2021*new_weights_2021[0],m=0)
current_investment
remain_spgi_2021=weight_change_fun(start='2021-10-12',end='2022-01-01',investment=current_investment,m=0)
remain_spgi_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

12971.69

## VZ

In [106]:
remain_vz_2021=remain_vz_2020

## TSLA

In [107]:
remain_tsla_2021=weight_change_fun(start='2021-03-03',end='2021-10-15',investment=new_investment_2021*new_weights_2021[2],m=2)
remain_tsla_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


18080.23

## PG

In [108]:
remain_pg_2021=weight_change_fun(start='2021-01-18',end='2021-03-27',investment=new_investment_2021*new_weights_2021[3],m=3)
remain_pg_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13680.53

## DIS

In [109]:
remain_dis_2021=weight_change_fun(start='2021-05-12',end='2022-01-01',investment=new_investment_2021*new_weights_2021[4],m=4)
remain_dis_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13368.89

## SYY

In [110]:
remain_syy_2021=weight_change_fun(start='2021-06-15',end='2022-01-01',investment=new_investment_2021*new_weights_2021[5],m=5)
remain_syy_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13668.83

## AAPL

In [111]:
remain_appl_2021=weight_change_fun(start='2021-02-23',end='2021-07-01',investment=new_investment_2021*new_weights_2021[6],m=6)
remain_appl_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


14972.82

## XOM

In [112]:
remain_xom_2021=weight_change_fun(start='2021-07-17',end='2021-10-21',investment=new_investment_2021*new_weights_2021[7],m=7)
remain_xom_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13254.99

## BRK-B

In [113]:
remain_brkb_2021=remain_brkb_2020

## UNH

In [114]:
current_investment=weight_change_fun(start='2020-12-31',end='2021-3-20',investment=new_investment_2021*new_weights_2021[9],m=9)
current_investment
remain_unh_2021=weight_change_fun(start='2021-10-02',end='2021-10-22',investment=current_investment,m=9)
remain_unh_2021

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

16845.51

## return and update weight

In [115]:
# previous return for 2020
com_pl_2021=[remain_spgi_2021,
            remain_vz_2021,
            remain_tsla_2021,
            remain_pg_2021,
            remain_dis_2021,
            remain_syy_2021,
            remain_appl_2021,
            remain_xom_2021,
            remain_brkb_2021,
            remain_unh_2021]
com_pl_2021

[12971.69,
 12992.15,
 18080.23,
 13680.53,
 13368.89,
 13668.83,
 14972.82,
 13254.99,
 10713.31,
 16845.51]

In [116]:
total_return_2021=sum(com_pl_2021)
total_return_2021

140548.94999999998

In [117]:
rcom_2021=[]
for i in range(10):
  rcom_2021.append(round((com_pl_2021[i]-com_pl_2020[i])/com_pl_2020[i],4))

rcom_2021

[-0.1301, 0.0, 0.047, 0.0503, 0.2317, 0.0538, 0.0678, 0.0098, 0.0, 0.3546]

In [118]:
# weight changed by return percentage
new_weights_2022_1=[]
for i in range(10):
  new_weights_2022_1.append(new_weights_2021[i]*(1+rcom_2021[i]))
# rescaling
new_weights_2022=[]
for i in range(10):
  new_weights_2022.append(round(new_weights_2022_1[i]/sum(new_weights_2022_1),2))
new_weights_2022

[0.08, 0.09, 0.1, 0.1, 0.12, 0.1, 0.1, 0.09, 0.09, 0.13]

In [119]:
new_investment_2022=total_return_2021
print('the investment for year 2022 is',new_investment_2022)
rbegin2021=(total_return_2021-100000)/100000
print('the return from the begining to 2021 is',rbegin2021 )
r20202021=(total_return_2021-total_return_2020)/new_investment
print('the return for the year 2021 is',r20202021 )

the investment for year 2022 is 140548.94999999998
the return from the begining to 2021 is 0.40548949999999984
the return for the year 2021 is 0.07110077195701574


# 2022

## SPGI

In [120]:
current_investment=weight_change_fun(start='2021-12-31',end='2022-8-11',investment=new_investment_2022*new_weights_2022[0],m=0)
current_investment
remain_spgi_2022=weight_change_fun(start='2022-9-20',end='2022-11-28',investment=current_investment,m=0)
remain_spgi_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

10626.11

## VZ

In [121]:
remain_vz_2022=remain_vz_2021

## TSLA

In [122]:
current_investment=weight_change_fun(start='2022-05-10',end='2022-07-30',investment=new_investment_2022*new_weights_2022[2],m=2)
current_investment
remain_tsla_2022=weight_change_fun(start='2022-10-05',end='2022-11-29',investment=current_investment,m=2)
remain_tsla_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

14104.48

## PG

In [123]:
remain_pg_2022=weight_change_fun(start='2022-05-17',end='2022-11-23',investment=new_investment_2022*new_weights_2022[3],m=3)
remain_pg_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


14320.22

## DIS

In [124]:
remain_dis_2022=weight_change_fun(start='2022-8-09',end='2022-11-29',investment=new_investment_2022*new_weights_2022[4],m=4)
remain_dis_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


18599.03

## SYY

In [125]:
current_investment=weight_change_fun(start='2021-12-30',end='2022-02-16',investment=new_investment_2022*new_weights_2022[5],m=5)
current_investment
remain_syy_2022=weight_change_fun(start='2022-09-21',end='2022-10-29',investment=current_investment,m=5)
remain_syy_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

15768.16

## AAPL

In [126]:
remain_appl_2022=weight_change_fun(start='2022-09-28',end='2022-11-29',investment=new_investment_2022*new_weights_2022[6],m=6)
remain_appl_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


15055.81

## XOM

In [127]:
remain_xom_2022=weight_change_fun(start='2022-09-24',end='2022-10-29',investment=new_investment_2022*new_weights_2022[7],m=7)
remain_xom_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


15750.68

## BRK-B

In [128]:
remain_brkb_2022=weight_change_fun(start='2022-04-30',end='2022-11-23',investment=new_investment_2022*new_weights_2022[8],m=8)
remain_brkb_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


13602.83

## UNH

In [129]:
remain_unh_2022=weight_change_fun(start='2022-06-15',end='2022-11-01',investment=new_investment_2022*new_weights_2022[9],m=9)
remain_unh_2022

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


19401.76

## total return

In [130]:
com_pl_2022=[remain_spgi_2022,
            remain_vz_2022,
            remain_tsla_2022,
            remain_pg_2022,
            remain_dis_2022,
            remain_syy_2022,
            remain_appl_2022,
            remain_xom_2022,
            remain_brkb_2022,
            remain_unh_2022]
com_pl_2022

[10626.11,
 12992.15,
 14104.48,
 14320.22,
 18599.03,
 15768.16,
 15055.81,
 15750.68,
 13602.83,
 19401.76]

In [131]:
total_return_2022=sum(com_pl_2022)
total_return_2022

150221.23

In [132]:
rcom_2022=[]
for i in range(10):
  rcom_2022.append(round((com_pl_2022[i]-com_pl_2021[i])/com_pl_2021[i],4))

rcom_2022

[-0.1808, 0.0, -0.2199, 0.0468, 0.3912, 0.1536, 0.0055, 0.1883, 0.2697, 0.1517]

In [133]:
rbegin2022=(total_return_2022-100000)/100000
print('the return from the begining to 2021 is',rbegin2022 )
r20212022=(total_return_2022-total_return_2021)/total_return_2021
print('the return for the year 2021 is',r20202021 )

the return from the begining to 2021 is 0.5022123000000001
the return for the year 2021 is 0.07110077195701574
